In [5]:
import os
import sys
current_dir = '/home/yueyulin/github/RWKV_LM_EXT'
sys.path.append(current_dir)
print(f'add {current_dir} to sys.path')
os.environ['RWKV_JIT_ON'] = '0'
os.environ['RWKV_T_MAX'] = '4096'
os.environ['RWKV_FLOAT_MODE'] = 'bf16'
precision = "bf16"
os.environ['RWKV_HEAD_SIZE_A'] = '64'
os.environ['RWKV_T_MAX'] = '4096'
os.environ["RWKV_MY_TESTING"]='x060'
os.environ['RWKV_CTXLEN'] = '4096'

add /home/yueyulin/github/RWKV_LM_EXT to sys.path


In [8]:
def create_empty_args():
    import argparse
    args = argparse.Namespace()
    args.my_pos_emb = 0
    args.pre_ffn = 0
    args.head_size_divisor = 8
    args.dropout = 0
    args.head_qk = 0
    args.ctx_len = 4096
    args.grad_cp = 0
    return args

args = create_empty_args()
base_rwkv_model = '/media/yueyulin/KINGSTON/models/rwkv6/RWKV-x060-World-1B6-v2.1-20240328-ctx4096.pth'
from src.model_ext import load_ckpt_and_parse_args
w = load_ckpt_and_parse_args(base_rwkv_model, args)
print(w)
print(args)

OrderedDict([('emb.weight', tensor([[ 3.8281e-01, -2.1362e-02, -8.9355e-02,  ..., -7.8125e-02,
          3.1006e-02,  6.2256e-02],
        [ 1.0400e-01, -4.4336e-01,  1.9141e-01,  ...,  1.7969e-01,
          5.6152e-03, -3.5400e-02],
        [ 1.0840e-01,  4.2480e-02, -2.2070e-01,  ..., -5.0354e-03,
          1.7456e-02, -1.1865e-01],
        ...,
        [-1.1683e-04, -8.5354e-05, -1.4722e-05,  ..., -6.4373e-05,
         -1.8239e-05,  4.4823e-05],
        [-2.9683e-05, -2.8491e-05, -1.3530e-05,  ...,  1.5378e-05,
         -3.4094e-05,  5.3346e-06],
        [-9.7752e-05, -1.9550e-05,  1.1683e-05,  ...,  6.1512e-05,
          2.6822e-05,  3.1710e-05]], dtype=torch.bfloat16)), ('blocks.0.ln1.weight', tensor([0.7891, 1.3516, 1.0078,  ..., 0.8125, 1.0078, 1.1484],
       dtype=torch.bfloat16)), ('blocks.0.ln1.bias', tensor([ 0.2598, -0.0776, -0.0571,  ..., -0.0140,  0.0134,  0.0435],
       dtype=torch.bfloat16)), ('blocks.0.ln2.weight', tensor([1.2656, 1.1875, 1.0156,  ..., 0.7227, 0.8984

In [9]:
tokenizer_file = os.path.join(current_dir, 'tokenizer', 'rwkv_vocab_v20230424.txt')
from tokenizer.rwkv_tokenizer import TRIE_TOKENIZER
tokenizer = TRIE_TOKENIZER(tokenizer_file)
print(tokenizer)

In [16]:
input_strs = ['我们来自哪里','中国首都在北京']
input_ids = [tokenizer.encode(input_str)+[1] for input_str in input_strs]
print(input_ids)
max_len = max([len(input_id) for input_id in input_ids])
print(max_len)
for input_id in input_ids:
    input_id.extend([0]*(max_len-len(input_id)))
print(input_ids)
import torch
input_ids = torch.tensor(input_ids,dtype=torch.long,device='cuda')
print(input_ids)

[[12605, 10402, 13234, 15847, 11187, 17384, 1], [10285, 11435, 18117, 17311, 11454, 10902, 10362, 1]]
8
[[12605, 10402, 13234, 15847, 11187, 17384, 1, 0], [10285, 11435, 18117, 17311, 11454, 10902, 10362, 1]]
tensor([[12605, 10402, 13234, 15847, 11187, 17384,     1,     0],
        [10285, 11435, 18117, 17311, 11454, 10902, 10362,     1]],
       device='cuda:0')


In [17]:
from src.model_bi import RWKV
model = RWKV(args)
print(model)

RWKV_MY_TESTING x060


Using /home/yueyulin/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/yueyulin/.cache/torch_extensions/py310_cu121/wkv6/build.ninja...
Building extension module wkv6...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF wkv6_op.o.d -DTORCH_EXTENSION_NAME=wkv6 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/yueyulin/miniconda3/envs/pytorch-env/lib/python3.10/site-packages/torch/include -isystem /home/yueyulin/miniconda3/envs/pytorch-env/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/yueyulin/miniconda3/envs/pytorch-env/lib/python3.10/site-packages/torch/include/TH -isystem /home/yueyulin/miniconda3/envs/pytorch-env/lib/python3.10/site-packages/torch/include/THC -isystem /home/yueyulin/miniconda3/envs/pytorch-env/include -isystem /home/yueyulin/miniconda3/envs/pytorch-env/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/yueyulin/github/RWKV_LM_EXT/cuda/wkv6_op.cpp -o wkv6_op.o 
[2/3] /home/yueyulin/miniconda3/envs/pytorch-env/bin/nvcc  -DTORCH_EXTENSION_NAME=wkv6 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\"

Loading extension module wkv6...


RWKV(
  (emb): Embedding(65536, 2048)
  (blocks): ModuleList(
    (0): Block(
      (ln1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (ln0): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (att): RWKV_Tmix_x060(
        (time_shift): ZeroPad2d((0, 0, 1, -1))
        (receptance): Linear(in_features=2048, out_features=2048, bias=False)
        (key): Linear(in_features=2048, out_features=2048, bias=False)
        (value): Linear(in_features=2048, out_features=2048, bias=False)
        (output): Linear(in_features=2048, out_features=2048, bias=False)
        (gate): Linear(in_features=2048, out_features=2048, bias=False)
        (ln_x): GroupNorm(32, 2048, eps=0.00064, affine=True)
      )
      (ffn): RWKV_CMix_x060(
        (time_shift): ZeroPad2d((0, 0, 1, -1))
        (key): Linear(in_features=2048, out_features=7168, bias=False)
        (receptance): Linear(in_features=2048, out_featur

In [18]:
info = model.load_state_dict(w)
print(info)

<All keys matched successfully>


In [19]:
model = model.to(torch.bfloat16)
model = model.to('cuda')
input_ids = input_ids.to('cuda')
output = model(input_ids)
print(output)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)